<a href="https://colab.research.google.com/github/AleksiKnuutila/trumpvschapo/blob/master/Scattertext_comparing_r_The_Donald_to_r_ChapoTrapHouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install scattertext

     |████████████████████████████████| 6.8MB 2.8MB/s 


#### Simple directory structure

In [0]:
!mkdir external_data
!mkdir processed_data

#### Libraries

In [0]:
import scattertext
from spacy.lang.en import English
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


### Provide your credentials to the runtime

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


#### Access Reddit comments via BigQuery and save to CSV

In [0]:
%%bigquery --project oiiapp df
SELECT body,subreddit FROM `fh-bigquery.reddit_comments.2018_11` 
WHERE subreddit IN ('The_Donald','ChapoTrapHouse')

In [0]:
df.subreddit.value_counts()

The_Donald        987062
ChapoTrapHouse    315718
Name: subreddit, dtype: int64

In [0]:
df.to_csv('external_data/comments-2018_11.csv')


#### Tokenise and make scattertext visualisation, then save corpus

In [0]:
corpus = scattertext.CorpusFromPandas(df,category_col='subreddit',text_col='body',
                                      nlp=nlp).build().get_unigram_corpus()

html = scattertext.produce_scattertext_explorer(corpus,category='The_Donald',
                                                category_name='The Donald', not_category_name='Chapo', 
                                                minimum_term_frequency=1250, max_docs_per_category=0)

In [0]:
term_freq_df = corpus.get_term_freq_df()
for subreddit in ['The_Donald','ChapoTrapHouse']:
  term_freq_df['{} Score'.format(subreddit)] = corpus.get_scaled_f_scores(subreddit)

term_freq_df.to_csv('processed_data/term_freqs_and_scores-2018_11.csv')

In [0]:
open('scattertext-reddit.html', 'wb').write(html.encode('utf-8'))

984399